In [ ]:
import os 
import sys 

cur_path = os.path.abspath(".")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

from utils import remove_prefix, ddf 

load_dotenv()
sg = Subgrounds()
bs: Subgraph = sg.load_subgraph(os.environ['SUBGRAPH_URL'])
pool = bs.Query.metapoolOracles(first=10000, orderBy="season", orderDirection="asc")
df_pool_raw = sg.query_df(
    [
        pool.balanceA, 
        pool.balanceB, 
        pool.season, 
        pool.deltaB, 
    ],
    pagination_strategy=ShallowStrategy
)
df_pool = df_pool_raw.copy()
df_pool = remove_prefix(df_pool, "metapoolOracles_")
df_pool.balanceA /= 10**6
df_pool.balanceB /= 10**18
df_pool['diff'] = df_pool.balanceB - df_pool.balanceA
print(len(df_pool))
df_pool.tail()
nearest = alt.selection_single(
    nearest=True, encodings=["x"], on="mouseover", empty="none"
)
brush = alt.selection_interval(encodings=['x'])

base = alt.Chart(df_ftokens)
x = alt.X("humidity:Q")
y = alt.Y("fertilizer:Q")

# Histogram of fertilizer purchased by season 
histogram = base.mark_bar().encode(
    x=x, y=y
)

# Vertical line to indicate nearest histogram bin 
vline = base.mark_rule(color="grey").encode(
    x=x, size=alt.value(1)
).transform_filter(
    nearest
)

# Text label to indicate value nearest histogram bin 
text = histogram.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, y, alt.value(' '))
)

# Time axis which drives scale of histogram 
time_axis = base.mark_bar().encode(
    x=x, y=y
).add_selection(
    brush 
).resolve_scale(
    'independent'
).properties(height=50)

(
    alt.layer(
        histogram.add_selection(nearest), 
        text, 
        vline
    ).transform_filter(
        brush
    ).properties(height=250) 
    & time_axis 
) 
ferts = bs.Query.fertilizerBalances(first=10000)

df_ferts_raw = sg.query_df(
    [
        ferts.id, 
        ferts.amount, 
        ferts.farmer, 
        ferts.fertilizerToken, 
    ],
    pagination_strategy=ShallowStrategy
)
#.sort_values('podFills_transaction_timestamp').reset_index(drop=True)
df_ferts = df_ferts_raw.copy()
print(len(df_ferts))
print(df_ferts.fertilizerBalances_amount.sum())
print(df_ferts.fertilizerBalances_amount.sum())
print(df_ferts.columns)
df_ferts.head()
"""
fertilizer balances by season 
"""

df_ferts = df_ferts_raw.copy()
df_ferts = df_ferts.rename(columns={
    'fertilizerBalances_amount': 'amount',
    'fertilizerBalances_farmer_id': 'address', 
    'fertilizerBalances_fertilizerToken_humidity': 'humidity',
    'fertilizerBalances_fertilizerToken_season': 'season' # season fertilizer was minted in 
})
df_ferts = df_ferts[['amount', 'address', 'season', 'humidity']].sort_values("season")

# ddf(df_ferts)